In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

#### Read Power grid info

In [2]:
# Create buses
num_buses = 1354
buses = np.arange(1, num_buses+1)

# Read gen buses
file_path = '../../data/IEEE_Case1354/zones/gen_bus.csv'
gen_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read thermal buses
file_path = '../../data/IEEE_Case1354/zones/thermal_bus.csv'
thermal_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read wind gen buses
file_path = '../../data/IEEE_Case1354/zones/wind_bus.csv'
wind_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read load buses
file_path = '../../data/IEEE_Case1354/zones/load_bus.csv'
load_buses = pd.read_csv(file_path, header=None, index_col=None)

In [3]:
# Boolean index of thermal buses
thermal_bidx = np.isin(buses, thermal_buses)

# Boolean index of wind buses
wind_bidx = np.isin(buses, wind_buses)

# Boolean index of load buses
load_bidx = np.isin(buses, load_buses)

In [4]:
num_zones = 8

# Dynamic variable name
for i in range(1, num_zones+1):
    file_path = f'../../data/IEEE_Case1354/zones/zone{i}_thermal_buses.csv'
    locals()[f'zone{i}_thermal_buses'] = pd.read_csv(file_path, header=None, index_col=None)

    file_path = f'../../data/IEEE_Case1354/zones/zone{i}_load_bus.csv'
    locals()[f'zone{i}_load_buses'] = pd.read_csv(file_path, header=None, index_col=None)

In [5]:
for i in range(1, num_zones+1):
    locals()[f'zone{i}_thermal_bidx'] = np.isin(buses, locals()[f'zone{i}_thermal_buses'])
    locals()[f'zone{i}_load_bidx'] = np.isin(buses, locals()[f'zone{i}_load_buses'])

#### Create node features

In [6]:
nt = 12     # Time steps

num_samples = 1000  # Number of SCUC samples
max_num = 1528  # Max number

# Empty list to store data
x = []

# Assign wind and load features
node_features = np.zeros((num_buses, 2*nt))

for i in range(max_num):
    # File path
    wind_path = f'../../data/IEEE_Case1354/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case1354/inputs/load/sample_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, :nt] = wind.values
    node_features[load_bidx, -nt:] = load.values

    x.append(node_features.copy())

The file sample_1 is not found
The file sample_3 is not found
The file sample_5 is not found
The file sample_9 is not found
The file sample_11 is not found
The file sample_12 is not found
The file sample_14 is not found
The file sample_19 is not found
The file sample_20 is not found
The file sample_24 is not found
The file sample_28 is not found
The file sample_31 is not found
The file sample_35 is not found
The file sample_38 is not found
The file sample_41 is not found
The file sample_42 is not found
The file sample_44 is not found
The file sample_51 is not found
The file sample_54 is not found
The file sample_56 is not found
The file sample_59 is not found
The file sample_61 is not found
The file sample_63 is not found
The file sample_64 is not found
The file sample_71 is not found
The file sample_72 is not found
The file sample_74 is not found
The file sample_78 is not found
The file sample_79 is not found
The file sample_81 is not found
The file sample_84 is not found
The file sam

#### Create target features

In [10]:
# List to store target features
y = []

# Target features
agg_shedding = np.zeros((num_zones+1, nt))

temp = np.zeros((num_buses, nt))
temp2 = np.zeros((num_buses, nt))

for i in range(max_num):
    load_input_path = f'../../data/IEEE_Case1354/inputs/load/sample_{i+1}.csv'
    DL_path = f'../../data/IEEE_Case1354/outputs/deployed_load/sample_{i+1}.csv'
    try:
        load_input = pd.read_csv(load_input_path, header=None, index_col=None)
        DL = pd.read_csv(DL_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    temp[load_bidx] = load_input.values
    temp2[load_bidx] = DL.values

    agg_shedding[0, :] = temp.sum(axis=0) - temp2.sum(axis=0)
    for j in range(1, num_zones+1):
        agg_shedding[j, :] = temp[locals()[f'zone{j}_load_bidx']].sum(axis=0) - temp2[locals()[f'zone{j}_load_bidx']].sum(axis=0)

    y.append(agg_shedding.copy())

The file sample_1 is not found
The file sample_3 is not found
The file sample_5 is not found
The file sample_9 is not found
The file sample_11 is not found
The file sample_12 is not found
The file sample_14 is not found
The file sample_19 is not found
The file sample_20 is not found
The file sample_24 is not found
The file sample_28 is not found
The file sample_31 is not found
The file sample_35 is not found
The file sample_38 is not found
The file sample_41 is not found
The file sample_42 is not found
The file sample_44 is not found
The file sample_51 is not found
The file sample_54 is not found
The file sample_56 is not found
The file sample_59 is not found
The file sample_61 is not found
The file sample_63 is not found
The file sample_64 is not found
The file sample_71 is not found
The file sample_72 is not found
The file sample_74 is not found
The file sample_78 is not found
The file sample_79 is not found
The file sample_81 is not found
The file sample_84 is not found
The file sam

#### Create edge index and attributes

In [27]:
# Read branch info
file_path = '../../data/IEEE_Case1354/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Get branch index and attr
edge_index = branch[:, :2] - 1
edge_attr = branch[:, 2:]

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)
edge_attr = torch.from_numpy(edge_attr).float()

#### Create train, test and validation dataset

In [28]:
data_list = []

for i in range(len(x)):
    X = torch.from_numpy(x[i]).float()
    Y = torch.from_numpy(y[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

In [29]:
dir = f'../../train_val_test_dataset/IEEE_Case1354/agg_Shedding/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

from CustomDataset import CustomDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case1354/agg_Shedding/'
CustomDataset(root=root, data_list=data_list)

The old dataset has been deleted!


Processing...
Done!


CustomDataset(1000)